# Modelo Multimodal V3.0

## Importación de librerías 

In [ ]:
Importación de librerías

In [22]:
import pandas as pd
import os
import re
import numpy as np 
import xarray as xr
import cv2 
from tqdm import tqdm
from pyproj import Proj
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight

ModuleNotFoundError: No module named 'cv2'

## Importación de datos y fusión

En esta notebook se importarán los datos tabulares junto con las imágenes satelitales. 
Posteriormente, se procederá a la fusión de ambos datasets con el fin de obtener un único conjunto integrado, denominado df_multimodal, el cual será utilizado para el entrenamiento de la red neuronal convolucional (CNN).

En el siguiente fragmento de código se realizan tres pasos principales:

1) Carga de datasets
   * dataset_final_enriquecido.csv: contiene los datos tabulares.
   * imagenes_satelitales: conjunto de imágenes satelitales descargadas previamente.

2) Identificación de carpetas e imágenes
Cada carpeta es reconocida por sus 5 imágenes correspondientes, a partir de las cuales se extrae la fecha asociada al evento.

3) Integración de datos
Una vez obtenida la fecha de cada imagen, se agrupan y combinan la información proveniente de ambos datasets, generando un único dataset integrado que incluye tanto datos tabulares como imágenes satelitales.

In [3]:
print("--- Iniciando la preparación de datos para el Modelo Multimodal V3.0 ---")

# --- 1. Cargar nuestro dataset tabular completo ---
ruta_dataset = "../data/processed/dataset_final_enriquecido.csv"
try:
    df_completo = pd.read_csv(ruta_dataset, parse_dates=['date'])
    print(f"✅ Dataset tabular completo cargado con {len(df_completo)} filas.")
except FileNotFoundError:
    print(f"❌ ERROR: No se encontró el archivo '{ruta_dataset}'. Asegúrate de que exista.")
    df_completo = None

if df_completo is not None:
    # --- 2. Identificar las secuencias de imágenes que SÍ tenemos ---
    ruta_imagenes = "../data/raw/imagenes_satelitales/"
    
    # Escaneamos la carpeta de imágenes para obtener la lista de fechas que se descargaron.
    # Nos aseguramos de que cada carpeta tenga sus 5 imágenes.
    carpetas_completas = []
    if os.path.exists(ruta_imagenes):
        for carpeta in os.listdir(ruta_imagenes):
            ruta_carpeta = os.path.join(ruta_imagenes, carpeta)
            if os.path.isdir(ruta_carpeta) and len(os.listdir(ruta_carpeta)) >= 5:
                carpetas_completas.append(carpeta)
    
    # Extraemos solo la fecha (YYYY-MM-DD) del nombre de cada carpeta completa
    fechas_con_imagenes = [re.search(r"\d{4}-\d{2}-\d{2}", carpeta).group(0) for carpeta in carpetas_completas if re.search(r"\d{4}-\d{2}-\d{2}", carpeta)]
    fechas_con_imagenes = pd.to_datetime(fechas_con_imagenes)

    print(f"✅ Se encontraron {len(fechas_con_imagenes)} fechas con secuencias de imágenes completas.")

    # --- 3. Sincronizar: Filtrar el dataset tabular ---
    # Creamos nuestro DataFrame final, quedándonos solo con las filas de los días
    # para los cuales tenemos una secuencia de imágenes completa.
    df_multimodal = df_completo[df_completo['date'].isin(fechas_con_imagenes)].copy()

    print(f"\n✅ Dataset sincronizado. Ahora tenemos {len(df_multimodal)} filas listas para el modelo.")

    # --- 4. Verificación ---
    print("\n--- Muestra del Dataset para el Modelo V3.0 ---")
    display(df_multimodal.head())
    print("\n--- Conteo de eventos en el nuevo dataset ---")
    print(df_multimodal['granizo'].value_counts())

--- Iniciando la preparación de datos para el Modelo Multimodal V3.0 ---
✅ Dataset tabular completo cargado con 37399 filas.
✅ Se encontraron 101 fechas con secuencias de imágenes completas.

✅ Dataset sincronizado. Ahora tenemos 466 filas listas para el modelo.

--- Muestra del Dataset para el Modelo V3.0 ---


,date,station_name,PRCP,SNWD,TAVG,TMAX,TMIN,granizo,latitude,longitude,...,om_rain_sum,om_snowfall_sum,om_precipitation_hours,om_wind_gusts_10m_max,om_wind_direction_10m_dominant,om_shortwave_radiation_sum,om_et0_fao_evapotranspiration,om_dew_point_2m_mean,om_relative_humidity_2m_mean,om_pressure_msl_mean
24283,2017-03-25,"MALARGUE, AR",6.1,NaN,16.1,26.0,9.8,0,-35.48,-69.58,...,0.6,0.0,1.0,54.000000,105.97220,19.76,4.493612,5.172083,48.640656,1009.84160
24284,2017-03-25,"MENDOZA AERO, AR",0.0,NaN,23.2,31.3,NaN,0,-32.83,-68.78,...,0.0,0.0,0.0,38.160000,173.85895,20.04,4.896718,11.737751,49.119923,1007.72090
24285,2017-03-25,"SAN MARTIN, AR",NaN,NaN,23.3,33.0,16.4,0,-33.08,-68.48,...,2.1,0.0,5.0,38.519997,137.53076,20.21,4.709652,13.338666,55.882126,1008.12080
24286,2017-03-25,"SAN RAFAEL, AR",NaN,NaN,20.4,33.5,13.8,1,-34.58,-68.33,...,2.6,0.0,4.0,39.960000,357.63052,17.44,4.145039,12.633335,57.392117,1008.84590
24287,2017-03-26,"MALARGUE, AR",2.0,NaN,16.4,25.2,9.5,0,-35.48,-69.58,...,0.5,0.0,1.0,48.960000,86.02075,21.12,4.559690,4.172083,48.957110,1010.71674



--- Conteo de eventos en el nuevo dataset ---
granizo
0    417
1     49
Name: count, dtype: int64


Como se puede observar, los datos fueron cargados correctamente.
En cuanto a la diferencia entre clases, esta se debe a que existen fechas en las que en una estación se registró granizo y en otra no.

Lejos de ser un problema, esta situación puede aportar información valiosa al modelo, ya que le permite identificar el evento como específico de una localización determinada, contribuyendo así a una suerte de “geolocalización” del fenómeno.

Comprobación

In [10]:
df_multimodal.isnull().sum()

date                                0
station_name                        0
PRCP                              339
SNWD                              464
TAVG                                0
TMAX                              244
TMIN                               46
granizo                             0
latitude                            5
longitude                           5
om_weather_code                     5
om_temperature_2m_max               5
om_temperature_2m_min               5
om_apparent_temperature_mean        5
om_precipitation_sum                5
om_rain_sum                         5
om_snowfall_sum                     5
om_precipitation_hours              5
om_wind_gusts_10m_max               5
om_wind_direction_10m_dominant      5
om_shortwave_radiation_sum          5
om_et0_fao_evapotranspiration       5
om_dew_point_2m_mean                5
om_relative_humidity_2m_mean        5
om_pressure_msl_mean                5
dtype: int64

## Estrategia de imputación

Como se observó previamente, ocurre la misma situación que en el dataset obtenido a partir de NOAA.

Para resolver este inconveniente, se aplicará la misma estrategia utilizada anteriormente: imputación mediante el promedio entre los valores disponibles y los obtenidos a través de la API de Open-Meteo.
En los casos en los que no se disponga de datos adicionales, se utilizarán directamente los valores proporcionados por Open-Meteo.

El siguiente script implementa tres estrategias de imputación de datos:

1) Relleno con datos de Open-Meteo:
Los valores faltantes en el dataset de NOAA se completan utilizando la información obtenida desde la API de Open-Meteo.

2) Interpolación temporal:
Se aplica interpolación en función del tiempo para estimar valores intermedios en las series temporales.

3) Relleno residual con ceros:
En caso de que aún persistan valores faltantes, estos se reemplazan por 0, a fin de evitar la corrupción de la estructura temporal del dataset climatológico.

In [11]:
# df_multimodal
print("Iniciando limpieza final de NaN...")

# Estrategia 1
mapeo_columnas = {
    'TMAX': 'om_temperature_2m_max',
    'TMIN': 'om_temperature_2m_min',
    'TAVG': 'om_apparent_temperature_mean',
    'PRCP': 'om_precipitation_sum'
}

for col_noaa, col_om in mapeo_columnas.items():
    if col_noaa in df_multimodal.columns and col_om in df_multimodal.columns:
        df_multimodal[col_noaa] = df_multimodal[col_noaa].fillna(df_multimodal[col_om])
        print(f"  -> Huecos en '{col_noaa}' rellenados con datos de '{col_om}'.")

# Estrategia 2:
# Primero establecemos la fecha como índice.
df_multimodal = df_multimodal.set_index('date')
# Interpolamos.
df_multimodal = df_multimodal.interpolate(method='time')
# Reseteamos el índice para que 'date' vuelva a ser una columna.
df_multimodal = df_multimodal.reset_index()
print("  -> Huecos restantes rellenados con interpolación.")

# Estrategia 3: 
df_multimodal = df_multimodal.fillna(0)

print("\n✅ Limpieza final completada.")

# Verificamos
print("\n--- Información del Dataset 100% Limpio ---")
df_multimodal.info()

Iniciando limpieza final de NaN...
  -> Huecos en 'TMAX' rellenados con datos de 'om_temperature_2m_max'.
  -> Huecos en 'TMIN' rellenados con datos de 'om_temperature_2m_min'.
  -> Huecos en 'TAVG' rellenados con datos de 'om_apparent_temperature_mean'.
  -> Huecos en 'PRCP' rellenados con datos de 'om_precipitation_sum'.
  -> Huecos restantes rellenados con interpolación.

✅ Limpieza final completada.

--- Información del Dataset 100% Limpio ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466 entries, 0 to 465
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            466 non-null    datetime64[ns]
 1   station_name                    466 non-null    object        
 2   PRCP                            466 non-null    float64       
 3   SNWD                            466 non-null    float64       
 4   TAVG                          

C:\Users\matia\AppData\Local\Temp\ipykernel_10400\3017496548.py:21: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_multimodal = df_multimodal.interpolate(method='time')


## Preparación para el modelo

Este código se encargará de:
1) Seleccionar las características (X) y el objetivo (y).
2) Dividir los datos en conjuntos de entrenamiento y prueba.
3) Escalar las características para optimizar el entrenamiento.

1) Las Características de Tiempo Finales

In [18]:
print("Creando nuevas características...")
df_multimodal['año'] = df_multimodal['date'].dt.year
df_multimodal['mes'] = df_multimodal['date'].dt.month
df_multimodal['dia_del_año'] = df_multimodal['date'].dt.dayofyear
df_multimodal['rango_temp_diario'] = df_multimodal['TMAX'] - df_multimodal['TMIN']
print("✅ Nuevas características ('año', 'mes', etc.) creadas exitosamente.")

Creando nuevas características...
✅ Nuevas características ('año', 'mes', etc.) creadas exitosamente.


2) Los datos tabulares (X_tabular) y el objetivo (y)

In [19]:
y = df_multimodal['granizo']

# Seleccionamos todas las columnas numéricas como características,
# excluyendo las que no son predictivas o ya están representadas.
columnas_a_excluir = [
    'date', 'station_name', 'granizo', 'latitude', 'longitude', 'año',
    # Excluimos las 'om_' que ya usamos para rellenar y evitar redundancia
    'om_temperature_2m_max', 'om_temperature_2m_min', 
    'om_apparent_temperature_mean', 'om_precipitation_sum'
]
# Seleccionamos todas las columnas que no están en la lista de exclusión
X_tabular = df_multimodal.drop(columns=columnas_a_excluir)

print(f"✅ Datos tabulares separados. Usaremos {X_tabular.shape[1]} características numéricas.")

✅ Datos tabulares separados. Usaremos 19 características numéricas.


3) División de Entrenamiento y Prueba

In [20]:
# Necesitamos dividir los índices para poder seleccionar las imágenes correctas después
indices = df_multimodal.index
train_indices, test_indices, y_train, y_test = train_test_split(
    indices, y, test_size=0.2, random_state=42, stratify=y
)

# Ahora seleccionamos los datos tabulares usando los índices
X_tabular_train = X_tabular.loc[train_indices]
X_tabular_test = X_tabular.loc[test_indices]

print("\n✅ Datos divididos en conjuntos de entrenamiento y prueba.")


✅ Datos divididos en conjuntos de entrenamiento y prueba.


4) Escalado de Características Tabulares

In [21]:
scaler = StandardScaler()
X_tabular_train_scaled = scaler.fit_transform(X_tabular_train)
X_tabular_test_scaled = scaler.transform(X_tabular_test)
print("✅ Datos tabulares escalados exitosamente.")

✅ Datos tabulares escalados exitosamente.


##  Procesar y Dividir el Dataset de Imágenes

1) Preparación 

In [ ]:
#1
ruta_base_imagenes = "../data/raw/imagenes_satelitales/"
IMG_SIZE = 64 # Tamaño estándar para las imágenes

# Diccionario para guardar en caché los objetos de proyección y evitar recalcularlos
proj_cache = {}

2) Función para procesar UNA secuencia de 5 imágenes 

In [ ]:
#2
def procesar_secuencia(fecha, granizo_status):
    nombre_carpeta = f"{fecha.strftime('%Y-%m-%d')}_{('granizo' if granizo_status == 1 else 'no_granizo')}"
    ruta_carpeta = os.path.join(ruta_base_imagenes, nombre_carpeta)
    
    if not os.path.exists(ruta_carpeta):
        return None # Si la carpeta no existe, devolvemos Nada
        
    archivos_secuencia = sorted(os.listdir(ruta_carpeta))
    secuencia_procesada = []
    
    for nombre_archivo in archivos_secuencia[:5]: # Nos aseguramos de tomar solo 5
        ruta_completa = os.path.join(ruta_carpeta, nombre_archivo)
        ds = xr.open_dataset(ruta_completa)
        
        # Recorte geoespacial
        proj_info = ds.goes_imager_projection
        h_sat = proj_info.perspective_point_height
        lon_cen = proj_info.longitude_of_projection_origin
        
        # Usamos caché para el objeto Proj
        if lon_cen not in proj_cache:
            proj_cache[lon_cen] = Proj(proj='geos', h=h_sat, lon_0=lon_cen)
        p = proj_cache[lon_cen]
        
        x1, y1 = p(-70.5, -37.5); x2, y2 = p(-66.5, -32.0)
        ds['x'] = ds['x'] * h_sat; ds['y'] = ds['y'] * h_sat
        recorte = ds.sel(x=slice(x1, x2), y=slice(y2, y1))['CMI'].values
        
        # Redimensionar y Normalizar
        recorte_redimensionado = cv2.resize(recorte, (IMG_SIZE, IMG_SIZE))
        recorte_normalizado = (recorte_redimensionado - np.nanmin(recorte_redimensionado)) / (np.nanmax(recorte_redimensionado) - np.nanmin(recorte_redimensionado) + 1e-6)
        secuencia_procesada.append(np.nan_to_num(recorte_normalizado))
        
    # Apilamos las 5 imágenes para formar un array de (64, 64, 5)
    return np.stack(secuencia_procesada, axis=-1)

3) Procesar TODAS las imágenes

In [ ]:
#3
print("Iniciando procesamiento de todas las secuencias de imágenes... (esto puede tardar)")
X_images_full = [procesar_secuencia(row['date'], row['granizo']) for index, row in tqdm(df_multimodal.iterrows(), total=len(df_multimodal))]

# Convertimos la lista de imágenes a un único array de NumPy
X_images_full = np.array(X_images_full)
print(f"\n✅ Procesamiento de imágenes completo. Dimensiones: {X_images_full.shape}")

4) Dividir el dataset de imágenes usando los mismos índices que los datos tabulares

In [ ]:
X_images_train = X_images_full[train_indices]
X_images_test = X_images_full[test_indices]
print("\n✅ Dataset de imágenes dividido en conjuntos de entrenamiento y prueba.")
print(f"Dimensiones de X_images_train: {X_images_train.shape}")
print(f"Dimensiones de X_images_test: {X_images_test.shape}")

## Construir la Arquitectura Multimodal (V3.0)

1) Definir la Rama para los Datos Tabulares (MLP)
2) Definir la Rama para las Imágenes (CNN)
3) Unir las dos Ramas

In [ ]:
#1
# Esta parte del cerebro se especializa en leer los números.
input_tabular = keras.Input(shape=(X_tabular_train_scaled.shape[1],), name="input_numerico")
x = layers.Dense(32, activation="relu")(input_tabular)
x = layers.Dense(16, activation="relu")(x)
# Guardamos la salida de esta rama en una variable
rama_tabular_salida = layers.Dense(8, activation="relu")(x)


#2
# Esta parte del cerebro se especializa en "ver" las imágenes.
input_imagenes = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 5), name="input_visual") # 64x64 píxeles, 5 imágenes en secuencia
y = layers.Conv2D(filters=16, kernel_size=(3, 3), activation="relu")(input_imagenes)
y = layers.MaxPooling2D(pool_size=(2, 2))(y)
y = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(y)
y = layers.MaxPooling2D(pool_size=(2, 2))(y)
y = layers.Flatten()(y) # Aplanamos el resultado para poder unirlo
# Guardamos la salida de esta rama en una variable
rama_imagenes_salida = layers.Dense(16, activation="relu")(y)


#3
# Concatenamos las salidas de ambos "expertos"
ramas_unidas = layers.concatenate([rama_tabular_salida, rama_imagenes_salida])

4) Cabeza de Clasificación Final

In [ ]:
# Un cerebro final que toma la decisión basándose en la información combinada
z = layers.Dense(16, activation="relu")(ramas_unidas)
z = layers.Dropout(0.5)(z) # Usamos Dropout para regularizar
output_final = layers.Dense(1, activation="sigmoid", name="output_final")(z)


5) Crear y Compilar el Modelo Multimodal

In [ ]:
modelo_multimodal_v3 = keras.Model(
    inputs=[input_tabular, input_imagenes],
    outputs=[output_final],
    name="modelo_multimodal_v3"
)

modelo_multimodal_v3.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

print("✅ Modelo Multimodal V3.0 construido y compilado.")
modelo_multimodal_v3.summary()

## Entrenamiento del Modelo Multimodal

1) Calcular Pesos para Clases Desbalanceadas

In [ ]:
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = dict(enumerate(class_weights))
print(f"Pesos de clase calculados: {class_weights_dict}")


2) Callbacks (Prácticas Profesionales para el Entrenamiento)

In [ ]:
# EarlyStopping: Detiene el entrenamiento si el modelo deja de mejorar.
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# ModelCheckpoint: Guarda el mejor modelo encontrado durante el entrenamiento.
os.makedirs("../models/", exist_ok=True) # Creamos la carpeta de modelos si no existe
checkpoint_path = "../models/multimodal_v3_best.keras"
model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True, monitor='val_loss')

print(f"\n✅ Callbacks definidos. El mejor modelo se guardará en: '{checkpoint_path}'")

3) Entrenar el Modelo Multimodal

In [ ]:
print("\nIniciando entrenamiento del Modelo Multimodal V3.0...")

historial_v3 = modelo_multimodal_v3.fit(
    # Le pasamos los dos sets de datos de entrenamiento como una lista
    [X_tabular_train_scaled, X_images_train],
    y_train,
    epochs=100, # Le damos un número alto de épocas, EarlyStopping decidirá cuándo parar
    batch_size=32,
    # Le pasamos los dos sets de datos de validación
    validation_data=([X_tabular_test_scaled, X_images_test], y_test),
    class_weight=class_weights_dict,
    callbacks=[early_stopping, model_checkpoint], # Usamos nuestros callbacks
    verbose=1
)

print("\n✅ Entrenamiento finalizado.")